In [ ]:
# Testing Large Sim

In [ ]:
import itertools
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

from matplotlib import cm

In [ ]:
# Functions

def generate_students(rng, num_applicants, b_rate):

    b_num = round(num_applicants * b_rate)
    w_num = round(num_applicants - b_num)
    
    w_dist = 15 * rng.standard_normal(w_num) + 100
    b_dist = 15 * rng.standard_normal(b_num) + 100

    return w_dist, b_dist

def generate_bias(rng, num_applicants, b_rate, avg, sd):

    b_num = round(num_applicants * b_rate)
    
    bias = sd * rng.standard_normal(b_num) + avg

    return bias

def select_students_bonus(b_dist, w_dist, quota, bias, bonus):

    if quota == 0:
        quota = round((w_dist.size + b_dist.size) / 10) 
    
    w_race = np.full((w_dist.size), 0)
    b_race = np.full((b_dist.size), 1)
    
    b_dist_2d = np.stack((b_dist, b_dist - bias + bonus, b_race), axis=-1)
    w_dist_2d = np.stack((w_dist, w_dist, w_race), axis=-1)

    both_dist_2d = np.concatenate([b_dist_2d, w_dist_2d])
    both_dist_2d = both_dist_2d[both_dist_2d[:, 1].argsort()]
    both_dist_2d_true = both_dist_2d[both_dist_2d[:, 0].argsort()]

    both_accepted_2d = both_dist_2d[-quota:]
    both_accepted_2d_true = both_dist_2d_true[-quota:]

    total_IQ_loss = (
        np.average(np.transpose(both_accepted_2d_true)[0]) - 
        np.average(np.transpose(both_accepted_2d)[0])
    ) * quota
    
    return total_IQ_loss

def find_optimal_bonus(w_dist, b_dist, quota, bias, upper_bound, lower_bound, increment, digits):

    optimal_bonus = lower_bound
    lowest_IQ_loss = float('inf')

    for digit in range(digits):

        bonus_points_arr = np.arange(lower_bound, upper_bound, increment)
    
        for bonus in bonus_points_arr:
        
            IQ_loss = select_students_bonus(b_dist, w_dist, quota, bias, bonus)

            if IQ_loss < lowest_IQ_loss:
                lowest_IQ_loss = IQ_loss
                optimal_bonus = bonus

        lower_bound = optimal_bonus - increment
        upper_bound = optimal_bonus + increment
        increment = increment / 10

    return(optimal_bonus, lowest_IQ_loss)

def vary_sim_across(vars_dict):

    vars_list = list(itertools.product(*vars_dict.values()))

    rng = 0
    num_applicants = 1
    b_rate = 2
    quota = 3
    bias_avg = 4
    bias_sd = 5
    lower_bound = 6
    upper_bound = 7
    increment = 8
    digits = 9
    num_rounds = 10

    all_optimal_bonuses = []
    all_IQ_losses = []

    for vars in vars_list:
        
        optimal_bonuses = np.zeros(shape=vars[num_rounds])
        IQ_losses = np.zeros(shape=vars[num_rounds])
        
        for r in range(vars[num_rounds]):
                
            w_dist, b_dist = generate_students(
                vars[rng],
                vars[num_applicants],
                vars[b_rate])
                
            bias = generate_bias(
                vars[rng],
                vars[num_applicants], 
                vars[b_rate],
                vars[bias_avg],
                vars[bias_sd])
                
            this_optimal_bonus, this_IQ_loss = find_optimal_bonus(
                w_dist,
                b_dist,
                vars[quota],
                bias,
                vars[upper_bound],
                vars[lower_bound],
                vars[increment],
                vars[digits])
                    
            optimal_bonuses[r] = this_optimal_bonus
            IQ_losses[r] = this_IQ_loss

        this_average_optimal_bonus = round(np.average(optimal_bonuses),3)
        this_average_IQ_loss = round(np.average(IQ_losses),3)

        all_optimal_bonuses.append(this_average_optimal_bonus)
        all_IQ_losses.append(this_average_IQ_loss)

    results = np.column_stack((vars_list, all_optimal_bonuses, all_IQ_losses))
    
    return results


In [ ]:
# Estimate Optimal Coeficient

vars = {
    'rng':[np.random.default_rng(8222024112)],
    'num_applicants':[10_000],
    'b_rate':[0.5],
    'quota':np.arange(300,10_000,100), 
    'bias_avg':np.arange(0,20,0.2),
    'bias_sd':np.arange(0,20,0.2), # Equates to ~1 million sims

    'lower_bound':[-30],
    'upper_bound':[31],
    'increment':[10],
    'digits':[3],
    'num_rounds':[1]
}

results = vary_sim_across(vars)

quota = results.transpose()[3]
bias_avg = results.transpose()[4]
bias_sd = results.transpose()[5]

optimal_values = results.transpose()[11]
opt_m_avg = optimal_values - bias_avg

ar = quota / 10_000
logit_ar = [math.log((n)/(1-n), 10) for n in ar]
bias_variance = np.square(bias_sd)
var_x_lar = np.multiply(bias_variance, logit_ar) 

dataset = pd.DataFrame(
    {'OMA': opt_m_avg,
     'VXL': var_x_lar})
dataset = dataset.apply(pd.to_numeric)

model = smf.ols(
    formula='OMA ~ 0 + VXL',
    data=dataset)

ols_results = model.fit()

print(ols_results.summary())